---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    import re
    import pandas as pd
    import numpy as np
    #OPen file to read and copy one line at a time into df.
    with open('university_towns.txt','r') as f:
        university_towns_arr = []
        for line in f:
            if '[edit]' in line:
                state = line
            elif '(' in line:
                town = line
                university_towns_arr.append([state,town])
            else:
                town = line[:-1]
                university_towns_arr.append([state,town])
    university_towns_df = pd.DataFrame(data = university_towns_arr, columns = ['State','RegionName'])
    #Removing unnecessary data from RegionName and State columns using regex matching and substitution
    university_towns_df['RegionName'] = university_towns_df['RegionName'].apply(lambda x:re.sub('[\s][\(].*[\\n]','',x))
    university_towns_df['State'] = university_towns_df['State'].apply(lambda x:re.sub('[\[].*[\\n]','',x))
    return university_towns_df

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    #Skip first seven rows. They are not required. 
    gdp_data_all = pd.read_excel('gdplev.xls',skiprows = 7)
    #Renaming columns
    gdp_data_all.columns = ['Annual','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','Drop_1','Quarter','GDP_$Bn_To_Drop','GDP','Drop_2']
    #Retaining required column based on problem definition
    gdp_data_all = gdp_data_all.drop(['Annual','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','Drop_1','GDP_$Bn_To_Drop','Drop_2'], axis =1)
    #Only data after 2000 is required, others are not
    gdp_data = gdp_data_all.loc[gdp_data_all['Quarter'] >= '2000q1'].reset_index(drop = True)
    recession_qtr_list = []
    #Identifying the start of recession. 
    for i in range(len(gdp_data)):
        if (i < len(gdp_data)-2):
            if ((gdp_data.loc[(i + 1),'GDP'] < gdp_data.loc[(i),'GDP']) & (gdp_data.loc[(i + 2),'GDP'] < gdp_data.loc[(i +1),'GDP'])):
                recession_qtr_list.append(gdp_data.loc[(i + 1),'Quarter'])
    return recession_qtr_list[0]

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp_data_all = pd.read_excel('gdplev.xls',skiprows = 7)
    gdp_data_all.columns = ['Annual','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','Drop_1','Quarter','GDP_$Bn_To_Drop','GDP','Drop_2']
    gdp_data_all = gdp_data_all.drop(['Annual','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','Drop_1','GDP_$Bn_To_Drop','Drop_2'], axis =1)
    gdp_data = gdp_data_all.loc[gdp_data_all['Quarter'] >= '2000q1'].reset_index(drop = True)
    qtr = get_recession_start()
    recession_end = []
    for i in range(gdp_data.loc[gdp_data['Quarter'] == qtr].index[0],len(gdp_data)):
        if (i < len(gdp_data)-2):
            if ((gdp_data.loc[(i + 1),'GDP'] > gdp_data.loc[(i),'GDP']) & (gdp_data.loc[(i + 2),'GDP'] > gdp_data.loc[(i +1),'GDP'])):
                recession_end.append(gdp_data.loc[(i + 1),'Quarter'])
    return recession_end[1]

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp_data_all = pd.read_excel('gdplev.xls',skiprows = 7)
    gdp_data_all.columns = ['Annual','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','Drop_1','Quarter','GDP_$Bn_To_Drop','GDP','Drop_2']
    gdp_data_all = gdp_data_all.drop(['Annual','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','Drop_1','GDP_$Bn_To_Drop','Drop_2'], axis =1)
    gdp_data = gdp_data_all.loc[gdp_data_all['Quarter'] >= '2000q1'].reset_index(drop = True)
    qtr = get_recession_start()
    recession_bottom = []
    for i in range(gdp_data.loc[gdp_data['Quarter'] == qtr].index[0],len(gdp_data)):
        if (i < len(gdp_data)-2):
            if ((gdp_data.loc[(i + 1),'GDP'] > gdp_data.loc[(i),'GDP']) & (gdp_data.loc[(i + 2),'GDP'] > gdp_data.loc[(i +1),'GDP'])):
                recession_bottom.append(gdp_data.loc[(i),'Quarter'])
    return recession_bottom[0]

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    #Retaining only required columns
    housing_data_subset1 = housing_data.loc[:,'RegionName':'State']
    housing_data_subset2 = housing_data.loc[:,'2000-01':]
    housing_data_req_subset = pd.concat([housing_data_subset1,housing_data_subset2],axis = 1)
    #Creatinf a qtr month dictionary
    month_qtr_dic = {'01':'q1','02':'q1','03':'q1','04':'q2','05':'q2','06':'q2','07':'q3','08':'q3','09':'q3','10':'q4','11':'q4','12':'q4'}
    col_names_arr = ['RegionName','State']
    #Check last two values of every column name, which indicates the year. Map it to quarter from quarter month dict
    for col_name in housing_data_req_subset.columns[2:]:
        col_name = col_name.replace(col_name[-3:],month_qtr_dic[col_name[-2:]])
        col_names_arr.append(col_name)
    housing_data_req_subset.columns = col_names_arr
    #Chaning state names from states dict
    housing_data_req_subset['State'] = housing_data_req_subset['State'].apply(lambda x: states[x])
    housing_data_req_subset.set_index(['State','RegionName'], drop = True, inplace = True)
    #Vertical grouping can also take place, with axis = 1 to group by column names rather than row values
    housing_data_grouped = housing_data_req_subset.groupby(housing_data_req_subset.columns,axis =1).mean()
    return housing_data_grouped

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    university_towns = get_list_of_university_towns()
    university_towns['Flag'] = 1
    start_quarter = get_recession_start()
    bottom_quarter = get_recession_bottom()
    data_frame = convert_housing_data_to_quarters()
    data_frame_req = data_frame.iloc[:,[data_frame.columns.get_loc(start_quarter)-1,data_frame.columns.get_loc(bottom_quarter)]]
    #(price_ratio=recession_bottom/quarter_before_recession)
    data_frame_req['Price_Ratio'] = data_frame_req[data_frame_req.columns[1]]/data_frame_req[data_frame_req.columns[0]]
    data_frame_req = data_frame_req.reset_index()
    final_df = pd.merge(data_frame_req, university_towns, left_on = ['State','RegionName'], right_on = ['State','RegionName'], how = 'left')
    final_df['Flag'].fillna(0, inplace = True)
    uni_df = final_df.loc[final_df['Flag'] == 1].drop(['State','RegionName','2008q2',bottom_quarter,'Flag'],axis = 1).dropna()
    normal_df = final_df.loc[final_df['Flag'] == 0].drop(['State','RegionName','2008q2',bottom_quarter,'Flag'], axis = 1).dropna()
    p_val = ttest_ind(uni_df.values,normal_df.values).pvalue[0]
    if (p_val < 0.01):
        different = True
    else:
        different = False
    if uni_df.values.mean() > normal_df.values.mean():
        better = "university town"
    else:
        better = "non-university town"
    return (different,p_val,better)